<a href="https://colab.research.google.com/github/zealpatel1990/KDM_spring_2021_class/blob/main/ICP%204/source_code/KDM_ICP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting up PySpark in Colab

Spark is written in the Scala programming language and requires the Java Virtual Machine (JVM) to run. Therefore, our first task is to download Java

In [ ]:
!sudo apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Next, we will install Apache Spark 3.0.1 with Hadoop 2.7

In [ ]:
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

Now, we just need to unzip that folder.

In [ ]:
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

There is one last thing that we need to install and that is the findspark library. It will locate Spark on the system and import it as a regular library.

In [ ]:
!pip install -q findspark

Now that we have installed all the necessary dependencies in Colab, it is time to set the environment path. This will enable us to run Pyspark in the Colab environment.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

We need to locate Spark in the system. For that, we import findspark and use the findspark.init() method.

In [ ]:
import findspark
findspark.init()

If you want to know the location where Spark is installed, use findspark.find()

In [ ]:
findspark.find()

'/content/spark-3.0.1-bin-hadoop2.7'

Now, we can import SparkSession from pyspark.sql and create a SparkSession, which is the entry point to Spark.

You can give a name to the session using appName() and add some configurations with config() if you wish.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Finally, print the SparkSession variable.

In [ ]:
spark

In [ ]:
# load data from data.csv file

df = spark.read.csv("/content/data.csv", header=True, inferSchema= True)
df.show()


+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|  

# 1. Transformation Tasks

## a. Grouping data by InternetService

In [ ]:
#task 1
transform_1 = df.groupBy('InternetService').count()
print("Transformation 1: Grouping data based on InternetService")
transform_1.show()

Transformation 1: Grouping data based on the Contract feature
+---------------+-----+
|InternetService|count|
+---------------+-----+
|    Fiber optic| 3096|
|             No| 1526|
|            DSL| 2421|
+---------------+-----+



## b. Filtering data for monthlyCharges which are greater than 100 USD

In [ ]:
# task 2

transform_2 = df.filter(df['MonthlyCharges'] > 100)
print("Transformation 3: Filtering data based on MonthlyCharges which is greater than 100 USD")
transform_2.show(20)

Transformation 3: Filtering data based on MonthlyCharges which is greater than 100 USD
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+
|7892-POOKP|Female|            0|    Yes|        No|    28|         Yes|          Yes|  

## c. Combining transformation task: Filtering + grouping by 

In [ ]:
# task 3

# showing number of Internet services used when people pay more than 100 USD
transform_3 = df.filter(df['MonthlyCharges'] > 100).groupBy('InternetService').count()
print("Transformation 3: Filtering data based on MonthlyCharges which is greater than 100 USD")
transform_3.show()

# showing number of Contract used when people pay more than 100 USD
transform_3 = df.filter(df['MonthlyCharges'] > 100).groupBy('Contract').count()
print("Transformation 3: Filtering data based on MonthlyCharges which is greater than 100 USD")
transform_3.show()

Transformation 3: Filtering data based on MonthlyCharges which is greater than 100 USD
+---------------+-----+
|InternetService|count|
+---------------+-----+
|    Fiber optic|  902|
+---------------+-----+

Transformation 3: Filtering data based on MonthlyCharges which is greater than 100 USD
+--------------+-----+
|      Contract|count|
+--------------+-----+
|Month-to-month|  324|
|      One year|  273|
|      Two year|  305|
+--------------+-----+



# 2. Actions Tasks 

## a. Action:  get last 3 element using tail function

In [ ]:
# action task 1
action_1 = df.tail(3)
print("Action 1: Getting last 3 data points from data set")
action_1


Action 1: Getting 3 data points from data set


[Row(customerID='4801-JZAZL', gender='Female', SeniorCitizen=0, Partner='Yes', Dependents='Yes', tenure=11, PhoneService='No', MultipleLines='No phone service', InternetService='DSL', OnlineSecurity='Yes', OnlineBackup='No', DeviceProtection='No', TechSupport='No', StreamingTV='No', StreamingMovies='No', Contract='Month-to-month', PaperlessBilling='Yes', PaymentMethod='Electronic check', MonthlyCharges=29.6, TotalCharges='346.45', Churn='No'),
 Row(customerID='8361-LTMKD', gender='Male', SeniorCitizen=1, Partner='Yes', Dependents='No', tenure=4, PhoneService='Yes', MultipleLines='Yes', InternetService='Fiber optic', OnlineSecurity='No', OnlineBackup='No', DeviceProtection='No', TechSupport='No', StreamingTV='No', StreamingMovies='No', Contract='Month-to-month', PaperlessBilling='Yes', PaymentMethod='Mailed check', MonthlyCharges=74.4, TotalCharges='306.6', Churn='Yes'),
 Row(customerID='3186-AJIEK', gender='Male', SeniorCitizen=0, Partner='No', Dependents='No', tenure=66, PhoneService=

## b. Action: Printing summary

In [ ]:
# action task 2
action_2 = df.summary()
print("Action 2: Getting all the summary of data set")
action_2.show()


Action 2: Getting last 3 data points from data set
+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|summary|customerID|gender|     SeniorCitizen|Partner|Dependents|            tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|    MonthlyCharges|      TotalCharges|Churn|
+-------+----------+------+------------------+-------+----------+------------------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+------------------+------------------+-----+
|  count|      7043

## c. Action: Printing first element

In [ ]:
# action Task3

action_3 = df.first()
print("Action 3: Showing first element of data set")
action_3

Action 3: Showing first element of data set


Row(customerID='7590-VHVEG', gender='Female', SeniorCitizen=0, Partner='Yes', Dependents='No', tenure=1, PhoneService='No', MultipleLines='No phone service', InternetService='DSL', OnlineSecurity='No', OnlineBackup='Yes', DeviceProtection='No', TechSupport='No', StreamingTV='No', StreamingMovies='No', Contract='Month-to-month', PaperlessBilling='Yes', PaymentMethod='Electronic check', MonthlyCharges=29.85, TotalCharges='29.85', Churn='No')